In [3]:
# fix_chart_paths.py — run from anywhere
from pathlib import Path
from datetime import datetime
import json, re

INSIGHTS = Path("/Users/ousmane/Desktop/2025 - Documentations/Python/3 Economics/genesis-research/app/data/insights.json")

data = json.loads(INSIGHTS.read_text(encoding="utf-8"))

def to_web_path(chart_path: str) -> str:
    if not chart_path:
        return chart_path
    # If it already starts with /charts/, keep it
    if chart_path.startswith("/charts/"):
        return chart_path
    # If it's an absolute or repo-ish path containing /public/charts/..., convert to /charts/...
    m = re.search(r"/public/(charts/.*)$", chart_path)
    if m:
        return "/" + m.group(1)
    # Also handle cases like "genesis-research/public/charts/..."
    m = re.search(r"public/(charts/.*)$", chart_path)
    if m:
        return "/" + m.group(1)
    # Fallback: if it ends with charts/... assume that is the tail
    m = re.search(r"(charts/.*)$", chart_path)
    if m:
        return "/" + m.group(1)
    # otherwise leave as-is
    return chart_path

for ins in data:
    cp = ins.get("chartPath","")
    fixed = to_web_path(cp)
    if fixed != cp:
        ins["chartPath"] = fixed

# sort by date ASC and re-id
def key(ins):
    try:
        return datetime.strptime(ins.get("date","01-01-1900"), "%d-%m-%Y")
    except Exception:
        return datetime(1900,1,1)

data.sort(key=key)
for i, ins in enumerate(data, 1):
    ins["id"] = i

# backup + write
backup = INSIGHTS.with_suffix(".json.bak-chartpaths")
backup.write_text(INSIGHTS.read_text(encoding="utf-8"), encoding="utf-8")
INSIGHTS.write_text(json.dumps(data, indent=2, ensure_ascii=False), encoding="utf-8")

print("✅ Fixed chartPath fields to web paths.")
print(f"🛟 Backup at: {backup}")
print(f"💾 Updated:   {INSIGHTS}")


✅ Fixed chartPath fields to web paths.
🛟 Backup at: /Users/ousmane/Desktop/2025 - Documentations/Python/3 Economics/genesis-research/app/data/insights.json.bak-chartpaths
💾 Updated:   /Users/ousmane/Desktop/2025 - Documentations/Python/3 Economics/genesis-research/app/data/insights.json


In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Slugify all chart filenames under public/charts and update app/data/insights.json.
- Converts filenames to safe ASCII hyphen slugs: "<slug-title>_YYYY-MM-DD.svg"
- Fixes spaces/Unicode/dashes/quotes, normalizes dates (YYYY-MM-DD)
- Resolves naming collisions with -v2, -v3, ...
- Updates insights.json chartPath to "/charts/<Category>/<new_filename>.svg"
- Repairs any bad paths (absolute local or repo paths) to web paths
- Creates a timestamped backup of insights.json
- Uses `git mv` when available (falls back to os.rename)
"""

from pathlib import Path
from datetime import datetime
import json, re, unicodedata, subprocess, shutil, sys

# === CONFIG ===
BASE_DIR = Path("/Users/ousmane/Desktop/2025 - Documentations/Python/3 Economics/genesis-research")
CHARTS_ROOT = BASE_DIR / "public" / "charts"
INSIGHTS_JSON = BASE_DIR / "app" / "data" / "insights.json"

# Set to True to actually rename files and write insights.json
APPLY_CHANGES = True  # <-- change to True when you're ready

# Colors (not modified here, only used for sanity if needed)
CATEGORY_FOLDERS = ["Equities","Economics","Technical","Commodities","Fixed Income","Currencies","Cross Assets","Education"]

# === Helpers ===

DASH_MAP = {"\u2010":"-","\u2011":"-","\u2012":"-","\u2013":"-","\u2014":"-","\u2212":"-"}
QUOTE_MAP = {"\u2018":"'", "\u2019":"'", "\u201C":'"', "\u201D":'"'}
SPACE_RE = re.compile(r"\s+")
NON_ALNUM_RE = re.compile(r"[^a-z0-9]+")

def nfkd(s: str) -> str:
    return unicodedata.normalize("NFKD", s or "")

def ascii_strip(s: str) -> str:
    # NFKD then drop accents
    return "".join(c for c in nfkd(s) if not unicodedata.combining(c))

def tidy_text(s: str) -> str:
    s = ascii_strip(s)
    for k,v in {**DASH_MAP, **QUOTE_MAP}.items():
        s = s.replace(k, v)
    s = SPACE_RE.sub(" ", s).strip()
    return s

def slugify_title(title: str) -> str:
    t = tidy_text(title).lower()
    t = NON_ALNUM_RE.sub("-", t)
    t = re.sub(r"-{2,}", "-", t).strip("-")
    return t or "chart"

def normalize_date_in_stem(stem: str) -> tuple[str, str]:
    """
    Try to split 'Title_YYYY-MM-DD' or 'Title_YYYY_MM_DD' -> (Title, YYYY-MM-DD)
    If no date detected, returns (stem, None)
    """
    m = re.match(r"^(.*)_([0-9]{4})[-_]?([0-9]{2})[-_]?([0-9]{2})$", stem)
    if not m:
        return stem, None
    title_part = m.group(1).strip()
    yyyy, mm, dd = m.group(2), m.group(3), m.group(4)
    try:
        dt = datetime(int(yyyy), int(mm), int(dd))
        return title_part, dt.strftime("%Y-%m-%d")
    except Exception:
        return title_part, None

def find_category_for_path(path: Path) -> str | None:
    try:
        rel = path.relative_to(CHARTS_ROOT)
    except ValueError:
        return None
    if rel.parts:
        cat = rel.parts[0]
        return cat if cat in CATEGORY_FOLDERS else None
    return None

def web_path(category: str, filename: str) -> str:
    return f"/charts/{category}/{filename}"

def run_git_mv(src: Path, dst: Path) -> bool:
    """Use `git mv -f` if available, return True if succeeded."""
    try:
        subprocess.run(["git", "--version"], cwd=BASE_DIR, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
        subprocess.run(["git", "mv", "-f", str(src), str(dst)], cwd=BASE_DIR, check=True)
        return True
    except Exception:
        return False

def safe_rename(src: Path, dst: Path):
    dst.parent.mkdir(parents=True, exist_ok=True)
    # handle case-insensitive FS weirdness: if names differ only by case/punct, do temp hop
    if src.resolve() == dst.resolve():
        return
    try_git = run_git_mv(src, dst)
    if not try_git:
        dst.parent.mkdir(parents=True, exist_ok=True)
        src.rename(dst)

# === Scan and plan renames ===

if not CHARTS_ROOT.exists():
    sys.exit(f"Charts root not found: {CHARTS_ROOT}")
if not INSIGHTS_JSON.exists():
    sys.exit(f"insights.json not found: {INSIGHTS_JSON}")

all_svg = sorted(CHARTS_ROOT.rglob("*.svg"))
print(f"📊 Found {len(all_svg)} chart file(s) to check under {CHARTS_ROOT}")

rename_plan = {}  # old_path -> new_path
occupied = set()  # set of (category, filename) to avoid collisions

for f in all_svg:
    category = find_category_for_path(f)
    if not category:
        # file outside known category; skip
        continue
    stem = f.stem
    title_part, date_part = normalize_date_in_stem(stem)

    # If we didn’t parse a date, we’ll keep the stem slug and leave out the date
    slug_title = slugify_title(title_part if title_part else stem)

    if date_part:
        new_name_base = f"{slug_title}_{date_part}"
    else:
        new_name_base = slug_title  # no date parsed; rare case

    # ensure .svg ext
    new_name = f"{new_name_base}.svg"

    # resolve collisions
    v = 2
    while (category, new_name) in occupied or ((f.parent / new_name) != f and (f.parent / new_name).exists()):
        new_name = f"{new_name_base}-v{v}.svg"
        v += 1

    occupied.add((category, new_name))

    new_path = f.parent / new_name

    # If identical (already slugified), skip; else plan rename
    if new_path != f:
        rename_plan[f] = new_path

print(f"🛠️ Planned renames: {len(rename_plan)}")
for src, dst in list(rename_plan.items())[:20]:
    print(f"   - {src.relative_to(BASE_DIR)}  →  {dst.relative_to(BASE_DIR)}")
if len(rename_plan) > 20:
    print(f"   … and {len(rename_plan)-20} more")

# === Apply renames ===
if APPLY_CHANGES and rename_plan:
    for src, dst in rename_plan.items():
        safe_rename(src, dst)
    print("✅ Renames applied.")
else:
    if rename_plan:
        print("ℹ️ Dry-run: set APPLY_CHANGES = True to perform renames.")

# Build a mapping from *old web path* to *new web path* for insights update
path_map = {}
for old, new in rename_plan.items():
    old_cat = find_category_for_path(old)
    new_cat = find_category_for_path(new)
    if old_cat and new_cat:
        path_map[web_path(old_cat, old.name)] = web_path(new_cat, new.name)

# Also, create a map from *current filesystem* for already-ok files (no rename)
for f in all_svg:
    cat = find_category_for_path(f)
    if not cat: continue
    p = web_path(cat, f.name)
    path_map.setdefault(p, p)  # identity

# === Update insights.json ===

def to_web_path_any(chart_path: str) -> str:
    """Convert any absolute/repo path to /charts/<...> web path (best effort)."""
    if not chart_path:
        return chart_path
    cp = chart_path
    if cp.startswith("/charts/"):
        return cp
    m = re.search(r"/public/(charts/.*)$", cp)
    if m: return "/" + m.group(1)
    m = re.search(r"(?:^|/)(charts/.*)$", cp)
    if m: return "/" + m.group(1)
    # nothing matched; keep as-is
    return cp

insights = json.loads(INSIGHTS_JSON.read_text(encoding="utf-8"))
if not isinstance(insights, list):
    sys.exit("insights.json is not a list")

updated = 0
fixed_format = 0

for ins in insights:
    old_cp_raw = ins.get("chartPath","")
    norm_old_web = to_web_path_any(old_cp_raw)

    # If we reformatted (e.g., removed absolute path), note it
    if norm_old_web != old_cp_raw:
        fixed_format += 1

    # If we renamed, swap to new path; else keep normalized/formatted
    new_cp = path_map.get(norm_old_web, norm_old_web)

    if ins.get("chartPath") != new_cp:
        ins["chartPath"] = new_cp
        updated += 1

# sort by date (dd-mm-YYYY) and re-id
def sort_key(ins):
    try:
        return datetime.strptime(ins.get("date","01-01-1900"), "%d-%m-%Y")
    except Exception:
        return datetime(1900,1,1)

insights.sort(key=sort_key)
for i, ins in enumerate(insights, 1):
    ins["id"] = i

print(f"🧩 insights.json paths updated: {updated} (reformatted: {fixed_format})")

if APPLY_CHANGES:
    # backup then write
    backup = INSIGHTS_JSON.with_suffix(".json.bak-slugify")
    backup.write_text(INSIGHTS_JSON.read_text(encoding="utf-8"), encoding="utf-8")
    INSIGHTS_JSON.write_text(json.dumps(insights, indent=2, ensure_ascii=False), encoding="utf-8")
    print(f"🛟 Backup: {backup}")
    print(f"💾 Updated: {INSIGHTS_JSON}")
else:
    print("ℹ️ Dry-run: set APPLY_CHANGES = True to write insights.json.")

print("\n✅ NEXT:")
print("1) Set APPLY_CHANGES = True and re-run to apply changes.")
print("2) git add -A && git commit -m 'Slugify charts + update insights paths' && git pull --rebase && git push")
print("3) Redeploy; filenames are now ASCII-hyphen and paths are web-safe.")


📊 Found 15 chart file(s) to check under /Users/ousmane/Desktop/2025 - Documentations/Python/3 Economics/genesis-research/public/charts
🛠️ Planned renames: 15
   - public/charts/Commodities/Gold Miners: From Capitulation to Conviction_2025-09-30.svg  →  public/charts/Commodities/gold-miners-from-capitulation-to-conviction_2025-09-30.svg
   - public/charts/Commodities/Gold and Geopolitics: From Relief to Resilience_2025-10-11.svg  →  public/charts/Commodities/gold-and-geopolitics-from-relief-to-resilience_2025-10-11.svg
   - public/charts/Economics/Confidence Slips, Momentum Risks Ahead_2025-10-10.svg  →  public/charts/Economics/confidence-slips-momentum-risks-ahead_2025-10-10.svg
   - public/charts/Economics/Inflation Psychology: Relief at 1-Year, Steady at 5-Year_2025-10-10.svg  →  public/charts/Economics/inflation-psychology-relief-at-1-year-steady-at-5-year_2025-10-10.svg
   - public/charts/Economics/Manufacturing Regains Momentum, Inflation Pressures Persist_2025-10-15.svg  →  publi